# Digikala data for Apple, Samsung, Xiaomi

In [ ]:
! pip install selenium
! pip install pandas==1.5.3
! pip install unidecode

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from unidecode import unidecode
import time

In [3]:
# this ignores the FutureWarning warning from pandas.append() 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### opening webdriver

In [4]:
driver = webdriver.Chrome()

In [5]:
url = "https://www.digikala.com/search/category-mobile-phone/"
apple_url = url + "apple/"
xiaomi_url = url + "xiaomi/"
samsung_url = url + "samsung/"

getting links of 200 apple product if availible

In [6]:
driver.maximize_window()
driver.get(apple_url)

In [26]:
# choose most selled
driver.find_element(By.XPATH, """/html/body/div[1]/div[1]/div[3]/div[3]/div[3]/section[1]/div[1]/div/div/div/div[2]/span[3]""").click()

In [27]:
apple_links = []
# link_count = how many data you want
link_count = 200
y = 480

for _ in range(link_count // 4):
    cards = driver.find_elements(By.XPATH, f"/html/body/div[1]/div[1]/div[3]/div[3]/div[3]/section[1]/div[2]/div/a")
    for card in cards:
        if "ناموجود" not in card.text and "توقف تولید" not in card.text and "به‌زودی" not in card.text:
            if (link := card.get_attribute("href")) not in apple_links:
                apple_links.append(link)

    driver.execute_script(f"window.scrollTo(0,{y})")
    y += 480
    time.sleep(2)

In [75]:
len(apple_links)

31

getting data from each link

In [86]:
apple_df = pd.DataFrame(columns=["brand", 'model', 'off_price',"price", "storage", "ram", "color", "link"])
apple_df

,brand,model,off_price,price,storage,ram,color,link


In [ ]:
i = 1
for link in apple_links:
    # print(i)
    # i += 1
    driver.get(link)
    time.sleep(3)
    # Loading all the page
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0,0)")
    time.sleep(2)

    wait = WebDriverWait(driver, 10)
    # getting the title
    title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, """h1.text-h4"""))).text

    # color
    try:
        # selecting first color
        color = driver.find_element(By.CSS_SELECTOR, """div.styles_InfoSectionVariationColor__pX_3M:nth-child(1)""")
        color.click()
        color = color.text
    except:
        color = "NAN"

    more = wait.until(EC.visibility_of_element_located((By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div/div[1]/section/span/span""")))
    try:
        more.location_once_scrolled_into_view
        # driver.execute_script(f"window.scrollTo({more.location['x']},{more.location['y']-300})")
        time.sleep(3)
        more.click()
    except:
        driver.execute_script(f"window.scrollTo({more.location['x']},{more.location['y']-300})")
        time.sleep(3)    
        more.click()


    # getting infos from specs
    ram = "NAN"
    storage = "NAN"
    model = "NAN"

    infos = driver.find_elements(By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div/div[1]/section/div[2]/div/div""")

    for info in infos:
        if "حافظه داخلی" in info.text:
            storage_info = infos.index(info)
            storage = unidecode(infos[storage_info].text.split("\n")[1].split(" ")[0])

        if "مقدار RAM" in info.text:
            ram_info = infos.index(info)
            ram = infos[ram_info].text.split("\n")[1]

        if "مدل" in info.text:
            model_info = infos.index(info)
            model = unidecode(infos[model_info].text.split("\n")[1])

    
    # Get price
    try:
        prices = driver.find_elements(By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div/div[2]/div/div[1]/div""")
        price_infos = []
        for info in prices:
            if "افزودن به سبد" in info.text:
                price_infos = info.text.split("\n")

        if len(price_infos) > 2:
            # with off
            price = unidecode(price_infos[0].replace(",", ""))
            price = int(price)
            try:
                off_price = unidecode(price_infos[2].replace(",", ""))
                off_price = int(off_price)
            except ValueError:
                off_price = "NAN"
        else:
            price = unidecode(price_infos[0].replace(",", ""))
            price = int(price)
            off_price = "NAN"
    except:
        # Not availible
        continue
    
    data = {
        "brand" : "apple",
        "title" : title,
        "model" : model,
        'price' : price,
        "off_price" : off_price,
        "color" : color,
        "storage" : storage,
        "ram" : ram,
        "link" : link
    }
    
    apple_df = apple_df.append(data, ignore_index=True)


In [89]:
apple_df.to_csv("apple_data.csv")

getting links of 200 samsung product if availible

In [6]:
driver.maximize_window()
driver.get(samsung_url)

In [7]:
# choose most selled
driver.find_element(By.XPATH, """/html/body/div[1]/div[1]/div[3]/div[3]/div[3]/section[1]/div[1]/div/div/div/div[2]/span[3]""").click()

In [8]:
samsung_links = []
# link_count = how many data you want
link_count = 200
y = 480

for _ in range(link_count // 4):
    cards = driver.find_elements(By.XPATH, f"/html/body/div[1]/div[1]/div[3]/div[3]/div[3]/section[1]/div[2]/div/a")
    for card in cards:
        if "ناموجود" not in card.text and "توقف تولید" not in card.text and "به‌زودی" not in card.text:
            if (link := card.get_attribute("href")) not in samsung_links:
                samsung_links.append(link)

    driver.execute_script(f"window.scrollTo(0,{y})")
    y += 480
    time.sleep(2)

In [9]:
len(samsung_links)

107

getting data from each link

In [10]:
samsung_df = pd.DataFrame(columns=["brand", 'model', 'off_price',"price", "storage", "ram", "color", "link"])
samsung_df

,brand,model,off_price,price,storage,ram,color,link


In [11]:
i = 1
for link in samsung_links:
    # print(i)
    # i += 1
    driver.get(link)
    time.sleep(3)
    # Loading all the page
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0,0)")
    time.sleep(2)

    wait = WebDriverWait(driver, 10)
    # getting the title
    title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, """h1.text-h4"""))).text

    # color
    try:
        # selecting first color
        color = driver.find_element(By.CSS_SELECTOR, """div.styles_InfoSectionVariationColor__pX_3M:nth-child(1)""")
        color.click()
        color = color.text
    except:
        color = "NAN"

    more = wait.until(EC.visibility_of_element_located((By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div/div[1]/section/span/span""")))
    try:
        more.location_once_scrolled_into_view
        # driver.execute_script(f"window.scrollTo({more.location['x']},{more.location['y']-300})")
        time.sleep(3)
        more.click()
    except:
        driver.execute_script(f"window.scrollTo({more.location['x']},{more.location['y']-300})")
        time.sleep(3)    
        more.click()


    # getting infos from specs
    ram = "NAN"
    storage = "NAN"
    model = "NAN"

    infos = driver.find_elements(By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div/div[1]/section/div[2]/div/div""")

    for info in infos:
        if "حافظه داخلی" in info.text:
            storage_info = infos.index(info)
            storage = unidecode(infos[storage_info].text.split("\n")[1].split(" ")[0])

        if "مقدار RAM" in info.text:
            ram_info = infos.index(info)
            ram = infos[ram_info].text.split("\n")[1]

        if "مدل" in info.text:
            model_info = infos.index(info)
            model = unidecode(infos[model_info].text.split("\n")[1])

    
    # Get price
    try:
        prices = driver.find_elements(By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div/div[2]/div/div[1]/div""")
        price_infos = []
        for info in prices:
            if "افزودن به سبد" in info.text:
                price_infos = info.text.split("\n")

        if len(price_infos) > 2:
            # with off
            price = unidecode(price_infos[0].replace(",", ""))
            price = int(price)
            try:
                off_price = unidecode(price_infos[2].replace(",", ""))
                off_price = int(off_price)
            except ValueError:
                off_price = "NAN"
        else:
            price = unidecode(price_infos[0].replace(",", ""))
            price = int(price)
            off_price = "NAN"
    except:
        # Not availible
        continue
    
    data = {
        "brand" : "samsung",
        "title" : title,
        "model" : model,
        'price' : price,
        "off_price" : off_price,
        "color" : color,
        "storage" : storage,
        "ram" : ram,
        "link" : link
    }
    
    samsung_df = samsung_df.append(data, ignore_index=True)


In [13]:
samsung_df.to_csv("samsung_data.csv")